In [ ]:
# Cell 1: Cài đặt đồng bộ
!pip install -q -U torch torchvision torchaudio transformers peft datasets bitsandbytes trl accelerate pandas scikit-learn

## Local Inference on GPU 
Model page: https://huggingface.co/Qwen/Qwen3-4B-Instruct-2507

⚠️ If the generated code snippets do not work, please open an issue on either the [model repo](https://huggingface.co/Qwen/Qwen3-4B-Instruct-2507)
			and/or on [huggingface.js](https://github.com/huggingface/huggingface.js/blob/main/packages/tasks/src/model-libraries-snippets.ts) 🙏

In [ ]:
import os
import gc
import torch
import pandas as pd
import numpy as np
from tqdm import tqdm
from datasets import Dataset, DatasetDict

# Import Transformers
from transformers import (
    AutoTokenizer, 
    AutoModelForCausalLM, 
    BitsAndBytesConfig, 
    TrainingArguments
)

# Import PEFT & TRL
from peft import LoraConfig, PeftModel, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer
from sklearn.metrics import accuracy_score, classification_report

try:
    del model
    del trainer
    del tokenizer
except:
    pass
gc.collect()
torch.cuda.empty_cache()

# Environment config
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [ ]:
# Load model
MODEL_NAME = "Qwen/Qwen3-4B-Instruct-2507"
NEW_MODEL_NAME = "Qwen-ViANLI-Hallucination-Detection"

# Config data path
DATA_PATH = "/kaggle/input/processed/processed_data_3labels/ViANLI"
TRAIN_FILE = os.path.join(DATA_PATH, "train.csv")
DEV_FILE = os.path.join(DATA_PATH, "dev.csv")
TEST_FILE = os.path.join(DATA_PATH, "test.csv")

# Thiết lập device
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Kiểm tra xem import có thành công không
print("Transformers version:", torch.__version__)

In [ ]:
# --- COPY ĐÈ VÀO CELL 5 ---
def load_and_process_data(train_path, dev_path, test_path):
    # Read CSV
    df_train = pd.read_csv(train_path)
    df_dev = pd.read_csv(dev_path)
    df_test = pd.read_csv(test_path)

    # Prompt được thiết kế khớp với 2 nhãn của bạn
    def format_instruction(row):
        prompt = f"""You are a medical AI assistant. Your task is to determine the logical relationship between the following Context and Statement.
        
        Context: {row['sentence1']}
        Statement: {row['sentence2']}

        Based on the context, classify the statement as one of the followings:
        - Entailment: The statement is definitely true based on the context.
        - Extrinsic-hal: The truth of the statement cannot be determined from the context (it might be true or false, but the context doesn’t say).
        - Intrinsic-hal: The statement is definitely false or contradicts the context.

        Answer:
        """
        return prompt

    def process_df(df):
        data = []
        for _, row in df.iterrows():
            # Kiểm tra dữ liệu rỗng
            if pd.isna(row['sentence1']) or pd.isna(row['sentence2']) or pd.isna(row['label']):
                continue
            
            user_content = format_instruction(row)
            
            # --- QUAN TRỌNG: Lấy trực tiếp nhãn text từ file CSV ---
            # Xóa khoảng trắng thừa nếu có
            assistant_content = str(row['label']).strip() 

            messages = [
                {"role": "user", "content": user_content},
                {"role": "assistant", "content": assistant_content}
            ]

            data.append({"messages": messages, "label_str": assistant_content})
        return Dataset.from_list(data)

    print("Processing Train set...")
    train_dataset = process_df(df_train)
    print("Processing Dev set...")
    dev_dataset = process_df(df_dev)
    print("Processing Test set...")
    test_dataset = process_df(df_test)

    return DatasetDict({"train": train_dataset, "dev": dev_dataset, "test": test_dataset})

# Load data
dataset = load_and_process_data(TRAIN_FILE, DEV_FILE, TEST_FILE)
print("\nSample train data:", dataset['train'][0])
# Kiểm tra xem label_str in ra có đúng là "Entailment" hoặc "Hallucination" không

In [ ]:
# Config Quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16, 
    bnb_4bit_use_double_quant=False,
)

# Load Tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# Load Model (Ép kiểu float16 ngay từ đầu)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    torch_dtype=torch.float16 
)

model.config.use_cache = False 
model.config.pretraining_tp = 1
model = prepare_model_for_kbit_training(model)

# LoRA Config
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=16, 
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]
)

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=8,
    optim="paged_adamw_32bit",
    save_steps=100,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=True,   
    bf16=False, 
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="cosine",
    report_to="none",
    gradient_checkpointing=True 
)

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset['train'],
    eval_dataset=dataset['dev'],
    peft_config=peft_config,
    args=training_args,
    processing_class=tokenizer,
)

# --- BƯỚC VÁ LỖI (HYBRID CAST) ---
print("Applying Hybrid Cast for T4 compatibility...")
for name, param in trainer.model.named_parameters():
    if param.requires_grad:
        if param.dtype != torch.float32:
            param.data = param.data.to(torch.float32)
    elif param.dtype == torch.bfloat16:
        param.data = param.data.to(torch.float16)

for name, buffer in trainer.model.named_buffers():
    if buffer.dtype == torch.bfloat16:
        buffer.data = buffer.data.to(torch.float16)
print("Hybrid Cast complete.")

# Train
trainer.train()

# Save Model Local
trainer.model.save_pretrained(NEW_MODEL_NAME)
tokenizer.save_pretrained(NEW_MODEL_NAME)
print("Training finished and model saved.")

In [ ]:
# --- COPY ĐÈ VÀO CELL 8 ---
import gc
from sklearn.metrics import classification_report
import pandas as pd

# 1. Dọn dẹp bộ nhớ
try:
    del model
    del trainer
except:
    pass
gc.collect()
torch.cuda.empty_cache()

# 2. Định nghĩa đường dẫn local
ADAPTER_PATH = f"./{NEW_MODEL_NAME}"

# 3. Load Base Model
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False,
)

base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    torch_dtype=torch.float16
)

# 4. Load Adapter
print(f"Loading adapter from {ADAPTER_PATH}...")
model = PeftModel.from_pretrained(base_model, ADAPTER_PATH)
model.eval()

# 5. Predict Function
def predict(messages, model, tokenizer):
    user_prompt = messages[0]['content'] 
    text = tokenizer.apply_chat_template(
        [{"role": "user", "content": user_prompt}],
        tokenize=False,
        add_generation_prompt=True
    )
    inputs = tokenizer([text], return_tensors="pt").to("cuda")
    with torch.no_grad():
        outputs = model.generate(
            **inputs, 
            max_new_tokens=10, 
            pad_token_id=tokenizer.eos_token_id
        )
    generated_ids = outputs[0][inputs.input_ids.shape[1]:]
    return tokenizer.decode(generated_ids, skip_special_tokens=True).strip()

# 6. Run Evaluation
y_true = []
y_pred = []

print("Starting evaluation on Test set...")
test_subset = dataset['test'] 

for i in tqdm(range(len(test_subset))):
    sample = test_subset[i]
    # Lấy nhãn gốc từ file CSV (đã là Entailment/Hallucination)
    ground_truth = sample['label_str']
    messages = sample['messages']
    
    try:
        prediction = predict(messages, model, tokenizer)
    except Exception as e:
        print(f"Error: {e}")
        prediction = "unknown"
    
    y_true.append(ground_truth)
    y_pred.append(prediction)

# 7. Report (Chuẩn hóa để so sánh)
def normalize_label(label):
    # Chuyển hết về chữ thường để so sánh cho chắc chắn
    label = str(label).lower().strip()
    
    if "Entailment" in label:
        return "Entailment"
    if "Intrinsic-Hal" in label:
        return "Intrinsic-Hal"
    if "Extrinsic-Hal" in label:
        return "Extrinsic-Hal"
    return "unknown"

# Chuẩn hóa cả Ground Truth và Prediction về cùng format
y_true_clean = [normalize_label(y) for y in y_true]
y_pred_clean = [normalize_label(p) for p in y_pred]

# Lọc bỏ các mẫu lỗi (unknown)
valid_indices = [i for i, x in enumerate(y_pred_clean) if x != "unknown"]
y_true_final = [y_true_clean[i] for i in valid_indices]
y_pred_final = [y_pred_clean[i] for i in valid_indices]

print(f"\nSố mẫu hợp lệ: {len(y_true_final)}/{len(y_true)}")

target_names = ["Entailment", "Intrinsic-Hal", "Extrinsic-Hal"]

# Tạo báo cáo
report_dict = classification_report(
    y_true_final, 
    y_pred_final, 
    labels=target_names, 
    output_dict=True
)
df_report = pd.DataFrame(report_dict).transpose()

# Nhân 100 để ra phần trăm
cols_to_scale = ['precision', 'recall', 'f1-score']
df_report[cols_to_scale] = df_report[cols_to_scale] * 100
df_report['support'] = df_report['support'].astype(int)

print("\nClassification Report (Unit: %):")
print("-" * 60)
print(df_report.to_string(float_format="{:.3f}".format))
print("-" * 60)

# Lưu kết quả
df_result = pd.DataFrame({
    "Ground_Truth": y_true, 
    "Prediction_Raw": y_pred, 
    "Prediction_Clean": y_pred_clean
})
df_result.to_csv("submission_results_vianli.csv", index=False)
print("Results saved.")